In [1]:


import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
''
sc = pyspark.SparkContext(appName="nasdaq_comp_details_raw_parsed_cleaned")
spark = SparkSession(sc)
from pyspark.sql.types import StructField, StructType, StringType, LongType,DoubleType,ArrayType,FloatType, BooleanType,IntegerType
import pyspark.sql.functions as F
from pyspark.sql.functions import udf

In [2]:
from skainet_spark import Pipeline, transform, Input, Output, Metadata,ValidatedPipeline,assign_shortcuts



## Nasdaq Company Details - raw

In [3]:
@transform(spark,
    raw = Output('/nasdaq_comp_details/raw/nasdaq_comp_details_raw.parquet'),
    metadata = Metadata('/nasdaq_comp_details/raw/nasdaq_comp_details_metadata_raw.csv', spark)
)
def nasdaq_bi_comp_data(spark,raw,metadata):
    schema=metadata()
    df = (spark
          .read
          .format('json')
          .option('header','true')
          .schema(schema)
          .load('input/nasdaq_comp_details/*.json')
         )
    pipeline=Pipeline(df)
    pipeline.show_dimensions()
    pipeline.dataframe.show(10)
    pipeline.write(raw)


In [4]:
nasdaq_bi_comp_data(spark)

cols: 7 rows: 344
+------+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+
|ticker|        company_name|         description|            industry|              sector|     telephone|             address|
+------+--------------------+--------------------+--------------------+--------------------+--------------+--------------------+
|  aapl|Apple Inc. Common...|Apple (ticker sym...|Computer Manufact...|          Technology|(408) 996-1010|ONE APPLE PARK WA...|
|   chd|Church & Dwight C...|We were founded i...|Package Goods/Cos...|    Basic Industries|  609-806-1200|500 CHARLES EWING...|
|   tsn|Tyson Foods, Inc....|Tyson Foods, Inc....|   Meat/Poultry/Fish|Consumer Non-Dura...|  479-290-4000|2200 DON TYSON PA...|
|   eqr|Equity Residentia...|Equity Residentia...|Real Estate Inves...|   Consumer Services|    3129281178|EQUITY RESIDENTIA...|
|   rhi|Robert Half Inter...|Robert Half Inter...|Professional Serv...|        

## Nasdaq Company Details - Parsed

In [5]:

import json
import enum


@transform(spark,
    raw = Input('/nasdaq_comp_details/raw/nasdaq_comp_details_raw.parquet',spark),
    parsed_nq = Output('/nasdaq_comp_details/parsed/nasdaq_comp_details_parsed.parquet'),

)
def parsed_nq_comp_data(spark,raw,parsed_nq):
    cols_to_rename = {
        'ticker':'ticker_symbol',
    }
    pipel_nq = Pipeline(raw)
   
    SHORTCUT='_nq'
   
    pipel_nq = (pipel_nq
               .rename_columns(cols_to_rename)
               .transform(parse_address)
                .transform(parse_connection_number)
                .transform(parse_company_name)
                .transform(parse_to_processed_schema)
                .transform(parse_ticker)
                .drop_columns(['split_address','temp_postbox'])
                .transform(assign_shortcuts,SHORTCUT)
                .show_dimensions()
               )
    #pipel_nq.dataframe.select('state','country','city','ticker_symbol').where((F.col('split_address').isNull())).show(50,0)
    #pipel_nq.dataframe.select(F.col('company_name','telephone')).sort(F.desc('company_name')).show(500,0)
    #pipel_nq.dataframe.select(F.col('company_name'),F.col('*')).sort(F.desc('company_name')).show(500)
    #pipel_nq.dataframe.select('country','ticker_symbol','city','state','address').where(F.col('country').isin(['Bermuda','Hm12'])).show(10,0)
    
    #pipel_nq.dataframe.select('telephone_nq','ticker_symbol_nq','city_nq','address_nq').where(F.col('ticker_symbol_nq') == F.lit('ALLE')).show(10,0)
    
#     pipel_nq.dataframe.select(F.col('company_name_nq')).where(F.col('ticker_symbol_nq') == F.lit('MCHP')).show(1,0)
    
#     pipel_nq.dataframe.cache()
#     city_state_country = pipel_nq.dataframe.select('ticker_symbol_nq','city_nq','state_nq','country_nq')
#     sector_industry = pipel_nq.dataframe.select('ticker_symbol_nq','industry_nq','sector_nq')
#     print('Nasdaq - CSC (City, State,Country)')
#     pipel_csc = Pipeline(city_state_country)
#     pipel_csc.show_dimensions()
#     pipel_csc.dataframe.printSchema()
#     pipel_csc.dataframe.show(5)
    
#     print('Nasdaq - Sector Industry')
#     pipel_si = Pipeline(sector_industry)
#     pipel_si.show_dimensions()
#     pipel_si.dataframe.printSchema()
#     pipel_si.dataframe.show(5)
    
    print('Nasdaq - Company Details')
    pipel_nq = pipel_nq.drop_columns(['address_nq'])
    pipel_nq.show_dimensions()
    pipel_nq.dataframe.printSchema()
    pipel_nq.dataframe.show(10)

    pipel_nq.write(parsed_nq)
    

def parse_to_processed_schema(dataframe):
    dataframe = dataframe.withColumn('website_link',F.lit(None).cast(StringType()))
    dataframe = dataframe.withColumn('num_employees',F.lit(None).cast(IntegerType()))
    return dataframe
def parse_ticker(dataframe):
    dataframe = dataframe.withColumn('ticker_symbol', F.upper(F.col('ticker_symbol')))
    return dataframe

def parse_address(dataframe):
    dataframe = (dataframe.withColumn('split_address',
                                     F.reverse(F.split(F.col('address'),',')))
                .withColumn('country',F.trim(F.col('split_address')[0]))
                 
                .withColumn('state',F.when(F.col('country')==F.lit('United States of America'),
                                            F.trim(F.col('split_address')[2]))
                             .otherwise(F.lit(None)))
                 .withColumn('city',F.when(F.col('country')==F.lit('United States of America'),
                                           F.trim(F.col('split_address')[3]))
                             .otherwise(F.lit(None)))
                .withColumn('temp_postbox',F.trim(F.col('split_address')[1]))
                .withColumn('city',
                           F.when((F.col('country')==F.lit('Switzerland')) & (F.col('temp_postbox').rlike('CH-*')) ,F.trim(F.col('split_address')[2])
                                 ).otherwise(F.col('city')))
                .withColumn('city',
                            F.when((F.col('country') == F.lit('United Kingdom')) & (F.col('temp_postbox').rlike('[A-Z][A-Z][0-9]*')), F.trim(F.col('split_address')[2]) 
                                  ).otherwise(F.col('city'))
                           )
                .withColumn('city',
                            F.when((F.col('country') == F.lit('Ireland')) & (F.col('temp_postbox').rlike('DUBLIN*')), F.trim(F.col('split_address')[2]) 
                                  ).otherwise(F.col('city'))
                           )
                .withColumn('city',
                            F.when((F.col('country') == F.lit('Ireland')) & (F.col('temp_postbox').rlike('D[0-9]*')), F.trim(F.col('split_address')[2]) 
                                  ).otherwise(F.col('city'))
                           )
                .withColumn('city',
                            F.when((F.col('country') == F.lit('Ireland')) & (F.col('temp_postbox') == F.lit('00000') ), F.trim(F.col('split_address')[2]) 
                                  ).otherwise(F.col('city'))
                           )
                .withColumn('city',
                           F.when( (F.col('country') != F.lit('United States of America')) & 
                                  (F.col('city').isNull()) & 
                                  (F.col('temp_postbox').rlike('[0-9][0-9][0-9][0-9][0-9]')), F.trim(F.col('split_address')[3])
                                 ).otherwise(F.col('city')))
                .withColumn('state',
                           F.when( (F.col('country') != F.lit('United States of America')) & 
                                  (F.col('state').isNull()) & 
                                  (F.col('temp_postbox').rlike('[0-9][0-9][0-9][0-9][0-9]')) & 
                                  (~(F.col('temp_postbox').rlike('00000'))), F.trim(F.col('split_address')[2])
                                 ).otherwise(F.col('state')))
                .withColumn('country',
                                   F.when( (F.col('country') != F.lit('United States of America')) & 
                                          (F.col('state').isNotNull()), F.lit('United States of America')
                                 ).otherwise(F.col('country')))
                .withColumn('city',F.when(F.col('state')==F.lit('ATLANTA'),F.lit('ATLANTA')).otherwise(F.col('city')))
                .withColumn('state',F.when(F.col('state')==F.lit('ATLANTA'),F.lit('Georgia')).otherwise(F.col('state')))
                .withColumn('state',F.when(F.col('country').rlike('[0-9][0-9][0-9][0-9][0-9]'),
                                           F.trim(F.col('split_address')[1]))
                            .otherwise(F.col('state')))
                .withColumn('city',F.when(F.col('country').rlike('[0-9][0-9][0-9][0-9][0-9]'),
                                           F.trim(F.col('split_address')[2]))
                            .otherwise(F.col('city')))
                .withColumn('country',F.when(F.col('country').rlike('[0-9][0-9][0-9][0-9][0-9]'),
                                           F.lit('United States of America'))
                            .otherwise(F.col('country')))
                 .withColumn('city',F.when(F.col('country') == F.lit('Bermuda'), 
                                             F.trim(F.col('split_address')[2]))
                             .otherwise(F.col('city')))
                 .withColumn('country',F.when(F.col('country') == F.lit('Bermuda'), 
                                             F.lit('United Kingdom'))
                             .otherwise(F.col('country')))
                .withColumn('city',F.regexp_replace('city',r'[0-9]{1,}',''))
                .withColumn('city',F.trim(F.initcap('city')))
                .withColumn('state',F.trim(F.initcap('state')))
                .withColumn('country',F.trim(F.initcap('country')))
                )

                #split address part
    
    return dataframe
    
def parse_connection_number(dataframe):
    
#F.concat_ws('-',F.col('telephone').substr(0,3),F.col('telephone').substr(4,3),F.col('telephone').substr(8,4) )
    usa_telepone_format = '[0-9][0-9][0-9]-[0-9][0-9][0-9]-[0-9][0-9][0-9][0-9]'
    usa_telepone_format_long = '[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]'
    dataframe = (dataframe
                  .withColumn('telephone',F.regexp_replace(F.col('telephone'),'\(|\)',''))
                 
                 .withColumn('telephone',F.when((F.col('country') == F.lit('United States Of America')) & (F.col('telephone').rlike(usa_telepone_format_long)),
                                                 F.concat_ws('-',F.col('telephone').substr(0,3),F.col('telephone').substr(4,3),F.col('telephone').substr(7,5) )
                                                
                                               ).otherwise(F.col('telephone')))
                 .withColumn('telephone',F.when((F.col('country') == F.lit('United States Of America')) & 
                                                (F.col('telephone').contains(' ')), 
                                                F.regexp_replace(F.col('telephone'),' ','-'))
                             .otherwise(F.col('telephone')))
                 .withColumn('telephone',F.when((F.col('country') == F.lit('United States Of America')) & 
                                                (F.col('telephone').contains('.')), 
                                                F.regexp_replace(F.col('telephone'),'\.','-'))
                             .otherwise(F.col('telephone')))
                 .withColumn('telephone',F.when((F.col('country') == F.lit('United States Of America')) & 
                                               (~F.col('telephone').rlike(usa_telepone_format)),
                                               F.concat_ws('-',F.col('telephone').substr(0,3),F.col('telephone').substr(4,8))
                                               ).otherwise(F.col('telephone')))


                )
                
                
    return dataframe

def parse_company_name(dataframe):
    dataframe = dataframe.withColumn('company_name',F.lower(F.col('company_name')))
    dataframe = dataframe.withColumn('company_name',F.concat(F.col('company_name'),F.lit(' ')))
    dataframe = dataframe.withColumn('company_name',F.regexp_replace('company_name',',',''))
    dataframe = dataframe.withColumn('company_name',F.regexp_replace('company_name',' common stock',''))
    dataframe = dataframe.withColumn('company_name',F.regexp_replace('company_name',' inc[.]| inc ',' incorporated '))
    dataframe = dataframe.withColumn('company_name',F.regexp_replace('company_name',' corp[.]',' corporation '))
    dataframe = dataframe.withColumn('company_name',F.regexp_replace('company_name',' co[.]',' company'))
    dataframe = dataframe.withColumn('company_name',F.regexp_replace('company_name','  ',' '))
    dataframe = dataframe.withColumn('company_name',F.regexp_replace('company_name',' ordinary share',''))
    dataframe = dataframe.withColumn('company_name',F.regexp_replace('company_name','\([^)]*\)',''))
    dataframe = dataframe.withColumn('company_name',F.trim(F.col('company_name')))
    return dataframe

In [6]:
parsed_nq_comp_data(spark)

cols: 12 rows: 344
Nasdaq - Company Details
cols: 11 rows: 344
root
 |-- ticker_symbol_nq: string (nullable = true)
 |-- company_name_nq: string (nullable = true)
 |-- description_nq: string (nullable = true)
 |-- industry_nq: string (nullable = true)
 |-- sector_nq: string (nullable = true)
 |-- telephone_nq: string (nullable = true)
 |-- country_nq: string (nullable = true)
 |-- state_nq: string (nullable = true)
 |-- city_nq: string (nullable = true)
 |-- website_link_nq: string (nullable = true)
 |-- num_employees_nq: integer (nullable = true)

+----------------+--------------------+--------------------+--------------------+--------------------+------------+--------------------+----------+-----------+---------------+----------------+
|ticker_symbol_nq|     company_name_nq|      description_nq|         industry_nq|           sector_nq|telephone_nq|          country_nq|  state_nq|    city_nq|website_link_nq|num_employees_nq|
+----------------+--------------------+--------------------

## Nasdaq Company Details - Clean

In [7]:


@transform(spark,
    parsed = Input('/nasdaq_comp_details/parsed/nasdaq_comp_details_parsed.parquet', spark),
    metadata = Metadata('/nasdaq_comp_details/clean/nasdaq_comp_details_metadata_clean.csv', spark),
    clean = Output('/nasdaq_comp_details/clean/nasdaq_comp_details_clean.parquet'),
    clean_exception = Output('/nasdaq_comp_details/exception/nasdaq_comp_details_clean_exception.parquet')
)
def nasdaq_comp_details_clean(spark, parsed, metadata, clean, clean_exception):
    
    schema = metadata()
    
    print(schema.fieldNames())

    pipe = Pipeline(parsed)
    pipe = (pipe
            .show_dimensions()
           )
    
    validated_pipe = ValidatedPipeline(pipe, metadata)
    validated_pipe = (validated_pipe
                      #.add_validation(F.col('home_team_name') == 'Arsenal', 'column is null')
                      .validate()
                     )
    validated_pipe.write(clean, clean_exception)

In [8]:
nasdaq_comp_details_clean(spark)

['ticker_symbol_nq', 'company_name_nq', 'description_nq', 'industry_nq', 'sector_nq', 'telephone_nq', 'country_nq', 'state_nq', 'city_nq', 'website_link_nq', 'num_employees_nq']
cols: 11 rows: 344
Validated count: 344
Exception count: 0
